# London Fire Incidents Analysis

In [1]:
%matplotlib inline 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the data
import data_download

# data_dir = "LFB-data"
LFB_data = pd.read_csv("LFB Incident data - Datastore - with notional cost and UPRN from January 2009.csv")
LFB_data.head()

Starting download ...


100%|██████████| 104M/104M [31:29<00:00, 57.6kB/s]   




patool: Extracting archive.zip ...
patool: ... archive.zip extracted to `archive' (local file exists).

Extract complete


,235138081,01 Jan 2009,2009,00:00:37,0,Special Service,Special Service.1,RTC,Road Vehicle,Car,...,319,Battersea.1,342,Clapham,2,2.1,2.2,1,255,1.1
0,1091,01 Jan 2009,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2091,01 Jan 2009,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,...,308.0,Edmonton,NaN,NaN,1.0,1.0,1.0,1.0,255.0,2.0
2,3091,01 Jan 2009,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),...,210.0,Hillingdon,NaN,NaN,1.0,1.0,1.0,1.0,255.0,2.0
3,5091,01 Jan 2009,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,...,233.0,Holloway,250.0,Holloway,1.0,2.0,2.0,1.0,255.0,1.0
4,6091,01 Jan 2009,2009,00:06:03,0,False Alarm,AFA,NaN,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,...,172.0,Kentish Town,222.0,Kentish Town,1.0,2.0,2.0,1.0,255.0,1.0


In [3]:
LFB_data.shape

(1489665, 39)

In [4]:
LFB_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489665 entries, 0 to 1489664
Data columns (total 39 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   235138081                              1489665 non-null  object 
 1   01 Jan 2009                            1489665 non-null  object 
 2   2009                                   1489665 non-null  int64  
 3   00:00:37                               1489665 non-null  object 
 4   0                                      1489665 non-null  int64  
 5   Special Service                        1489665 non-null  object 
 6   Special Service.1                      1489665 non-null  object 
 7   RTC                                    467694 non-null   object 
 8   Road Vehicle                           1489665 non-null  object 
 9   Car                                    1489665 non-null  object 
 10  In street close to gazetteer location  148

###### Lets Clean the data.

In [5]:
# Lets see if the data has missing values.
cols = LFB_data.columns[:]
colours = ['#000099', '#ffff00'] # specify the colours - yellow is missing. blue is not missing.
sns.heatmap(LFB_data[cols].isnull(), cmap=sns.color_palette(colours))
print("Yellow - Missing Values\nBlue - Non Missing")

In [ ]:
# Dividing the data into numerical and categorical for easier cleaning
# select numeric columns
df_numeric = LFB_data.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)


In [ ]:
df_numeric

In [ ]:

# select non numeric columns
df_categorical = LFB_data.select_dtypes(exclude=[np.number])
non_numeric_cols = df_categorical.columns.values
print(non_numeric_cols)

In [ ]:
df_categorical.isnull().sum()

We can see that we have a couple of categorical columns with missing values

In [ ]:
df_categorical.head()

In [ ]:
df_categorical.value_counts()